In [ ]:
from scscp import SCSCPCLI as cli
from openmath import openmath as om

In [ ]:
client = cli("localhost", 26133)

In [ ]:
# construct the polynomial x2y3z + x4yz2 + xz
int_ring = om.OMSymbol("integers", "ring3")
sdmp_sym = om.OMSymbol("SDMP", "polyd")
term_sym = om.OMSymbol("term", "polyd")
poly_ring_sym = om.OMSymbol("poly_ring_d_named", "polyd")
dmp_sym = om.OMSymbol("DMP", "polyd")
list_sym = om.OMSymbol("list", "list1")

var1 = om.OMVariable("x")
var2 = om.OMVariable("y")
var3 = om.OMVariable("z")

int0 = om.OMInteger(0)
int1 = om.OMInteger(1)
int2 = om.OMInteger(2)
int3 = om.OMInteger(3)
int4 = om.OMInteger(4)

poly_ring = om.OMApplication(poly_ring_sym, [int_ring, var1, var2])
term1 = om.OMApplication(term_sym, [int1, int2, int3, int1])
term2 = om.OMApplication(term_sym, [int1, int4, int1, int2])
term3 = om.OMApplication(term_sym, [int1, int1, int0, int1])
sdmp = om.OMApplication(sdmp_sym, [term1, term2, term3])
poly = om.OMApplication(dmp_sym, [poly_ring, sdmp1])

In [ ]:
# calculate the orbit of the list of variables
var_list = om.OMAppication(list_sym, [var1, var2, var3])
perm_gp = client.heads.permgp1.symmetric_group([3])
var_orbit = client.heads.permgp1.orbit([var_list, perm_gp])

In [ ]:
# filter out the duplicates
for perm1 in var_orbit:
    args1 = [int_ring] ++ perm1.arguments
    ring1 = om.OMApplication(poly_ring_sym, args1)
    poly1 = om.OMApplication(dmp_sym, [ring1, sdmp])
    for perm2 in var_orbit:
        args2 = [int_ring] ++ perm2.arguments
        ring2 = om.OMApplication(poly_ring_sym, args2)
        poly2 = om.OMApplication(dmp_sym, [ring2, sdmp])
        if client.heads.relation1.equality([poly1, poly2]):
            var_orbit.remove(perm2)

In [ ]:
poly_orbit = []
for perm in var_orbit:
    args = [int_ring] ++ perm.arguments
    ring = om.OMApplication(poly_ring_sym, args)
    poly_orbit.append(om.OMApplication(dmp_sym, [ring, sdmp]))

In [ ]:
poly_orbit